In [ ]:
import cupy as cp
import matplotlib.pyplot as plt

from core import RNN, MSELoss

%matplotlib inline

In [ ]:
t = cp.linspace(0, 10, 101)
data = cp.sin(t) + cp.sin(2 * t) - 1/2

X = data[:-1].reshape(1, -1, 1) # (B, 100, input_dim) -> (1, 100, 1)
Y = data[1:].reshape(1, -1, 1) # (1, 100, 1)

plt.plot(data.get())
plt.show()

In [ ]:
def sample(model: RNN, start_val: float, length=200):
    # start with shape (1, 1, 1) as (B, seq_len, inp_dim)
    current_input = cp.array(start_val).reshape(1, -1, 1)
    
    h = None
    
    output_sequence = []
    
    for _ in range(length):
        out, h = model.forward(current_input, h) # Outputs (B, seq_len, out_dim)
        # we put in one char and give it its previous state (it works bc the predictions are done just via the state in each step,
        # we dont do backprop here so no info is lost)
        
        # Shorter: Notice that in forward pass we only need previous state, it doesnt matter whether it holds grad or nah.
        
        pred = out[0, 0, 0]
        output_sequence.append(pred)
        
        current_input = out
        
    return output_sequence
        

In [ ]:
model = RNN(1, 128, 1)
loss_fn = MSELoss()
learning_rate = 0.001
n_epochs = 200

In [ ]:
loss_history = []

for i in range(n_epochs):
    h_prev = None
    X_noisy = X + cp.random.normal(0, 0.01, X.shape) # Add a little bit of noise to training
    
    preds, _ = model.forward(X_noisy, h_prev)
    loss = loss_fn.forward(preds, Y)
    loss_history.append(loss)
    
    
    dpreds = loss_fn.backward()
    dinputs = model.backward(dpreds)
    
    model.step(learning_rate)
    
    
    if i % 100 == 0:
        print(f"Step: {i} | Loss: {loss}")

In [ ]:
plt.plot([i.get() for i in loss_history])
plt.show()

sample_sequence = sample(model, -1/2, 100)
plt.plot([i.get() for i in sample_sequence])
plt.show()